# transform_item_indents

A partir da função _transform_legislations o texto será retornado com os tokens de recuo criados.

Essa função padroniza as chamadas de alineas, identificando-as implicitamente caso sejam chamados após um inciso, de forma que:

- ST_INCISO_V, a seja substituído por ST_INCISO_V, ST_ALINEA_A


In [2]:
import re

ST_PREFIX = "ST_"

def _transform_legislations(
    text,
    prefix_result,
    re_legislation,
    is_multiple=False,
    identification=None,
    previous_token=None,
    separator=None,
):
    "Create custom tokens for legislation"

    def build_token(m):
        if m.group("ignored_expression"):
            result = m.group("ignored_expression")
        else:
            result = ""
        result = (
            result
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    def build_token_multi(m):
        result = (
            m.group("ignored_expression")
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    # replaces first occurrences
    text = re_legislation.sub(build_token, text)

    # replaces multiple occurrences, if exists
    if is_multiple:
        regex_legislation_multi = re.compile(
            fr"(?:(?P<ignored_expression>{previous_token}{separator}){identification})",
            flags=re.I,
        )
        while regex_legislation_multi.findall(text) != []:
            text = regex_legislation_multi.sub(build_token_multi, text)

    return text


In [3]:
def transform_item_indents(text, **kwargs):
    #Create custom tokens for indents contained in a item and not explicitly mentioned in the text

    prefix_result = "ALINEA_"
    previous_token = fr"{ST_PREFIX}{prefix_result}\w+"
    separator = r"\s*[,e]\s+"
    indent = (
        r"(?<!\w)[\"|\“|\”]?(?P<identification>[a|b]?[a-z])[\"|\“|\”]?(?!\w)"
    )
    re_legislation = re.compile(
        fr"(?:(?P<ignored_expression>{ST_PREFIX}INCISO_[MDCLXVI]+,\s+){indent})",
        flags=re.I,
    )

    return _transform_legislations(
        text,
        prefix_result,
        re_legislation,
        True,
        indent,
        previous_token,
        separator,
    )


In [4]:
entradas = """
  ST_INCISO_V, a, b, c, d e e
  ST_INCISO_x, “a”, "b", "c"
  art. 102, ST_PARAGRAFO_2, ST_INCISO_IV, “a” da Constituição Federal
"""

print("Entradas: ",entradas)

Entradas:  
  ST_INCISO_V, a, b, c, d e e
  ST_INCISO_x, “a”, "b", "c"
  art. 102, ST_PARAGRAFO_2, ST_INCISO_IV, “a” da Constituição Federal



In [5]:
print("Saida: ",transform_item_indents(entradas))

Saida:  
  ST_INCISO_V, ST_ALINEA_A, ST_ALINEA_B, ST_ALINEA_C, ST_ALINEA_D e ST_ALINEA_E
  ST_INCISO_x, ST_ALINEA_A, ST_ALINEA_B, ST_ALINEA_C
  art. 102, ST_PARAGRAFO_2, ST_INCISO_IV, ST_ALINEA_A da Constituição Federal



# DATASET VICTOR

In [6]:
import pandas as pd

test = pd.read_csv('./data/train_small.csv')
text = test['body']

In [7]:
print("Dados de entrada:\n",text)

Dados de entrada:
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


In [14]:
lista = []
for item in text:
    lista.append(transform_item_indents(item))
        

In [16]:
print("Dados de saida: \n", lista[561])

Dados de saida: 
 {"quais aplica regime LEI_10887 segundo qual proventos aposentadoria serão calculados com base média aritmética simples das maiores remunerações utilizadas como base para contribuições servidor aos regimes previdência que esteve vinculado correspondentes oitenta por cento todo período contributivo desde competência julho desde do início contribuição posterior àquela competência ARTIGO_1º caput para estes últimos portanto gdpst será considerada integralmente para fins cálculo aposentadoria da pensão medida que integra remuneração utilizada como base para cobrança contribuição previdenciária ora nobres ministros caso dos autos verifica claramente pelos documentos acostados que servidor recorrido enquadra previsão ARTIGO_5º inciso alínea razão pela qual somente possível incidência contribuição previdenciária sobre cinqüenta por cento valor máximo respectivo nível classe padrão tendo vista que somente tal percentual será incorporado aos proventos aposentadoria pensão dest